## Load latest headlines

In [4]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-05-04-13-00 +0000,wsj,AUD’s Role as Economic Shock Absorber Might Fade,https://www.wsj.com/finance/currencies/asian-c...
1,2025-06-05-03-49-26 +0000,nyt,Drone Attacks Are the New Front in War. Can NA...,https://www.nytimes.com/2025/06/04/world/europ...
2,2025-06-05-03-22-03 +0000,nyt,Trump Travel Order Bans People From 12 Countri...,https://www.nytimes.com/2025/06/04/us/politics...
3,2025-06-05-03-22-00 +0000,wsj,Judge Blocks Deportation of Boulder Suspect’s ...,https://www.wsj.com/us-news/judge-blocks-depor...
4,2025-06-05-03-04-09 +0000,nyt,U.S. Brings Back Guatemalan Wrongly Deported t...,https://www.nytimes.com/2025/06/04/us/politics...


## Calculate word frequencies

In [5]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-%M-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_19180\2305397166.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-%M-00')


,word,score
14,trump,46
86,south,18
9,new,12
183,canada,10
29,mexico,10


## Rank headlines by score

In [6]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
53,2025-06-04-07-25-13 +0000,nyt,South Korea’s New President Will Face Deep Div...,https://www.nytimes.com/2025/06/03/world/asia/...,112
144,2025-05-29-19-51-00 +0000,wsj,President Trump claims tariffs will make the U...,https://www.wsj.com/economy/trade/the-economic...,111
59,2025-06-04-00-30-59 +0000,nyt,Where ​South Korea’s New President​ Lee Jae-my...,https://www.nytimes.com/2025/06/03/world/asia/...,109
78,2025-06-03-17-56-00 +0000,wsj,President Trump’s tariffs threaten to crimp U....,https://www.wsj.com/economy/global/world-growt...,108
126,2025-05-31-03-14-00 +0000,wsj,President Trump said he would double tariffs o...,https://www.wsj.com/economy/trade/trump-is-pra...,96


## Next steps